In [1]:
import numpy as np
import math
from sympy import *

In [2]:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    
    From: https://stackoverflow.com/a/6802723/2460137
    """
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

# Unit vector or standard basis
v_unit_or_basis = np.asarray([[1,0,0],[0,1,0],[0,0,1]])
#rotate around x
v_ar_x = (np.dot(rotation_matrix([1,0,0],(65/180.0)*np.pi), v_unit_or_basis))
#rotate around y
v_ar_x_y = (np.dot(rotation_matrix([0,1,0],(7.5/180.0)*np.pi), v_ar_x)) 
#rotate around z
v_ar_x_y_z = (np.dot(rotation_matrix([0,0,1],(5/180.0)*np.pi), v_ar_x_y)) 
print v_ar_x_y_z
#name as transformed basis
n = v_ar_x_y_z

[[ 0.98767211  0.08101314  0.13394277]
 [ 0.08641011  0.43132033 -0.89805126]
 [-0.13052619  0.8985542   0.4190027 ]]


In [3]:
#rotated basis
n1 = n[0] 
n2 = n[1] 
n3 = n[2]
#Unrotated (Static) basis
e = np.asarray([[1,0,0],[0,1,0],[0,0,1]])
e1 = e[0]
e2 = e[1]
e3 = e[2]

#u2 (from plane B) as projected onto the plane A, variables
u2xx, u2yy, u2zz = symbols('u2_xx, u2_yy, u2_zz')
#u2 vector in standard basis. V used for generality, but should be U2' for clarity
V = (u2xx,u2yy,u2zz)

In [4]:
def solve_v(V,u2x,u2y,u2z):
    """
    Return 'x,y,z' components of uknown vector V',
    given a vector with known components of (u2x, u2y, u2z).
    Solves the "directional cosine matrix"
    
    Based on: https://en.wikipedia.org/wiki/Euclidean_vector#Multiple_Cartesian_bases
    See below for extract from wiki
    """
    x = solve(Eq((u2x*e1 + u2y*e2 + u2z*e3).dot(n1),V[0]))
    y = solve(Eq((u2x*e1 + u2y*e2 + u2z*e3).dot(n2),V[1]))
    z = solve(Eq((u2x*e1 + u2y*e2 + u2z*e3).dot(n3),V[2]))
    return x,y,z

$u = p\mathbf{e}_1\cdot\mathbf{n}_1 + q\mathbf{e}_2\cdot\mathbf{n}_1 + r\mathbf{e}_3\cdot\mathbf{n}_1$,

$v = p\mathbf{e}_1\cdot\mathbf{n}_2 + q\mathbf{e}_2\cdot\mathbf{n}_2 + r\mathbf{e}_3\cdot\mathbf{n}_2$,

$w = p\mathbf{e}_1\cdot\mathbf{n}_3 + q\mathbf{e}_2\cdot\mathbf{n}_3 + r\mathbf{e}_3\cdot\mathbf{n}_3$


Rewriting in matrix form and changing u,v,w, p,q,r variable names:

$\begin{bmatrix} u2xx \\ u2yy \\ u2zz \\ \end{bmatrix}$ $= \begin{bmatrix} c_{11} & c_{12} & c_{13} \\ c_{21} & c_{22} & c_{23} \\ c_{31} & c_{32} & c_{33} \end{bmatrix}$ $\begin{bmatrix} u2x \\ u2y \\ u2z \end{bmatrix} $

In [5]:
def distance_norms(dU, N1, N2):
    """
    Return endpoints of the line of shortest distance between two
    normal vectors (N1, N2) multiplied by some factor (lam, mu) 
    originating from the end-points of vectors
    U1 and U2 sitting in two different planes, with a difference 
    a difference between, dU.
    Idea for "geometrical approach": https://math.stackexchange.com/a/2447276/60150
    Distance calce from: https://math.stackexchange.com/a/2213256/60150
    Note that square of vectors are replace by dot products.
    Notation changed: 
    L=a+bt => L=L1,a=u1, t=lam, b = N1
    M=c+ds => M=L2, c = u2_onto_A, s = mu, d = N2
    """
    A = -(N1.dot(N1) * N2.dot(N2) - (N1.dot(N2))**2)
    
    mu = (-(N1.dot(N1))*(N2.dot(dU)) + (N1.dot(dU))*(N2.dot(N1))) / (A)
    
    lam = ((N2.dot(N2))*(N1.dot(dU)) - (N1.dot(dU))*(N2.dot(N1))) / (A)
    
    #endpoint of line 1
    L1 = u1 + lam*N1 
    #endpoint of line 2
    L2 = u2_onto_A + mu*N2
    #distance SQUARED?!
    D = (dU + lam*N1 - mu*N2).dot(dU + lam*N1 - mu*N2)

    return L1, L2, D

In [6]:
U1 = Matrix([1.293,  -0.778, 0.0])
U2 = Matrix([1.18232290333500, -0.273702926146914, -0.964582946390880])
N1 = Matrix([0,0,1])
N2 = Matrix(n[2])

In [7]:
def distance_from_cross(U1,U2,N1,N2):
    """
    This and below MD cells
    From: https://math.stackexchange.com/a/2217845/60150
    """
    #coordinates of all points along the lines:
    #p1 = U1 + lam*N1
    #p2 = U2 + mu*N2
    N = N1.cross(N2)
    d = np.float64( N.dot((U1-U2)) )/np.sqrt(np.float64(N.dot(N)))
    return d

print distance_from_cross(U1,U2,N1,N2)

-0.037032983581


The coordinates of all the points along the lines are given by,r - points, e - direction vects

$$\begin{align} 
  \mathbf{p}_1 & = \mathbf{r}_1 + t_1 \mathbf{e}_1 \\
  \mathbf{p}_2 & = \mathbf{r}_2 + t_2 \mathbf{e}_2 \\
\end{align}$$


$$ d = \frac{ \mathbf{n}\cdot \mathbf{p}_1}{\|\mathbf{n}\|} - \frac{ \mathbf{n}\cdot \mathbf{p}_2}{\|\mathbf{n}\|} = \frac{ \mathbf{n} \cdot ( \mathbf{p}_1-\mathbf{p}_2)}{\| \mathbf{n} \|} = \frac{ \mathbf{n} \cdot ( \mathbf{r}_1-\mathbf{r}_2+t_1 \mathbf{e}_1 -t_2 \mathbf{e}_2)}{\| \mathbf{n} \|} $$

But since $\mathbf{n}\cdot \mathbf{e}_1 = \mathbf{n}\cdot \mathbf{e}_2 = 0$, the above is

$$ \boxed{ 
d = \frac{ \mathbf{n} \cdot ( \mathbf{r}_1-\mathbf{r}_2)}{\| \mathbf{n} \|}
}$$

In [9]:
uv_im19 = np.array([[  1.16 ,  -0.839,   0.   ,  10.   ,   6.2  ]])

uv_im20 = np.array([[  0.933,  -0.298, -10.   ,  10.   ,  -6.2  ]])

In [10]:
distance_list = []

for v in uv_im19:
    for u in uv_im20:
        #Assume that the vector U2  'xyz' = (u[0], u[1],0.0)
        u2x, u2y, u2z = u[0],u[1],0.0
        #Projecting U2 onto plane A
        u2_onto_A = Matrix([solve_v(V,u2x,u2y, u2z)[0][0],
                            solve_v(V,u2x,u2y, u2z)[1][0],
                            solve_v(V,u2x,u2y, u2z)[2][0]])
        print "u2_onto_A", u2_onto_A
        #Vector U1 'xyz' = (v[0],v[1],0.0)
        u1 = Matrix([v[0], v[1], 0.0])
        #I use the normal to plane A as n1z, for a standard basis that's (0 0 1)
        N1 = Matrix([0,0,1])
        #Similarly, using n2z
        N2 = Matrix(n[2])
        #print "\nN2:",N2
        #take the difference between u1 and the u2(projected onto A)
        dU = u1 - u2_onto_A
        #print "\ndU:", dU
        #L1,L2,D
        la, lb, dist = distance_norms(dU, N1, N2)
        #add the data to the list
        distance_list.append((v,u, la, lb, dist))

u2_onto_A Matrix([[0.897356166825033], [-0.0479128215210652], [-0.389550088428914]])


In [11]:
#just converting list to an array
arr_dist = np.asarray(distance_list)
print "closest approach info, U1,U2,L1,L2,D:\n", arr_dist[(np.where(arr_dist[:,4]==np.min(arr_dist[:,4])))][0]


closest approach info, U1,U2,L1,L2,D:
[array([  1.16 ,  -0.839,   0.   ,  10.   ,   6.2  ])
 array([  0.933,  -0.298, -10.   ,  10.   ,  -6.2  ])
 Matrix([
[              1.16],
[            -0.839],
[-0.274523852104903]])
 Matrix([
[  1.01532426452396],
[-0.860015953066002],
[-0.768239997879974]])
 0.265128371317798]


In [12]:
#calculating the plane B
#e = 0,0,1
#Looking at the angle between plane A and B, this seems to be the correct way to transform. 
#I just assume the translations are not crucial at this point (for geogebra plotting)
# From https://www.maplesoft.com/support/help/Maple/view.aspx?path=MathApps/EquationOfAPlaneNormal
np = (-0.13052619,  0.8985542,   0.4190027 )
p = (1.27, -0.889, 0.0)
x,y,z,d = symbols('x,y,z,d')
print 'coeficients for plane eq:', solve(x*np[0] + y*np[1] + z*np[2] + d)
#print p[0]*np[0],p[1]*np[1], p[2]*np[2] 
print 'solving for d:', solve(p[0]*np[0]+ p[1]*np[1]+ p[2]*np[2]+ d)

coeficients for plane eq: [{d: 0.13052619*x - 0.8985542*y - 0.4190027*z}]
solving for d: [0.964582945100000]
